In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_curve, auc, precision_score, recall_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import LocalOutlierFactor
from numpy import arange
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report

In [2]:
benign_df = pd.read_csv(r'D:\Kuliah\ProyekAkhir\New Data\resample\Benign_resample.csv')
benign_df.head(2)

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,15.854422,-0.343690,-0.086569,0.096351,0.142000,0.002723,0.706619,1.982467,1.798333,-0.034669,...,-0.059807,-0.095769,-0.109896,-0.112787,-0.020891,-0.339324,-0.282883,-0.341153,-0.315336,1
1,-0.060352,5.598377,2.253048,4.681794,3.829148,0.125465,3.569438,-0.039001,2.481753,5.125594,...,-0.059807,1.103561,0.162506,0.515708,1.673959,6.184780,12.942534,8.511326,2.479016,1


In [3]:
ddos_df = pd.read_csv(r'D:\Kuliah\ProyekAkhir\New Data\resample\DDoS_resample_300.csv')
ddos_df

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0.019544,-0.034901,-0.014359,-0.005940,-0.010963,-0.003071,0.010276,0.021431,0.019534,-0.009313,...,-0.014919,0.155863,-0.003031,-0.001923,-0.003225,-0.002593,-0.034464,-0.002979,-0.034487,-0.034441
1,-0.060352,6.721162,1.883858,2.601355,4.794288,0.509481,10.943103,-0.385539,3.799711,11.094226,...,4.940215,-0.059807,0.833868,0.444403,0.451340,0.675292,11.344368,-0.266215,8.505489,14.870348
2,2.209517,-0.327036,-0.309612,-0.339661,0.820881,-0.010439,2.527200,2.846555,2.732269,-0.198358,...,-0.366263,0.063244,-0.098072,-0.107270,-0.109441,-0.015768,-0.315335,-0.268473,-0.315664,-0.295924
3,-0.060352,-0.344934,-0.176134,0.096351,-0.068534,-0.003947,-0.175095,-0.385539,-0.365794,-0.034669,...,-0.472948,13.453306,-0.095769,-0.109896,-0.112787,-0.020891,-0.339324,-0.282883,-0.341153,-0.315336
4,15.854422,-0.343695,-0.086569,0.096351,0.357669,0.009473,1.609839,4.408229,4.015244,-0.034669,...,-0.226895,13.453306,-0.095769,-0.109896,-0.112787,-0.020891,-0.339324,-0.282883,-0.341153,-0.315336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,0.137589,-0.059109,-0.005918,-0.005398,-0.018964,-0.003128,0.010190,0.101722,0.089597,-0.054776,...,-0.005077,0.096986,-0.028542,-0.021380,-0.027278,-0.021925,-0.050755,-0.020652,-0.051561,-0.049015
407,15.854422,-0.343681,-0.086569,0.096351,0.157405,0.005133,0.771135,2.155736,1.956684,-0.034669,...,-0.226895,-0.059807,-0.095769,-0.109896,-0.112787,-0.020891,-0.339324,-0.282883,-0.341153,-0.315336
408,2.209517,-0.327036,-0.309612,-0.339661,0.820881,-0.010439,2.527200,2.846555,2.732269,-0.198358,...,-0.366263,0.063244,-0.098072,-0.107270,-0.109441,-0.015768,-0.315335,-0.268473,-0.315664,-0.295924
409,0.021072,-0.013958,-0.002917,-0.004205,1.235010,-0.001653,1.310350,1.311145,1.311242,-0.008236,...,-0.027228,-3.073348,-0.001168,-0.000797,-0.001254,-0.000998,-0.012431,-0.001560,-0.012326,-0.012412


In [34]:
ddos_df[' Label'] = -1

In [36]:
benign_test_df = pd.read_csv(r'D:\Kuliah\ProyekAkhir\New Data\resample\Benign_resample_4000.csv')

In [37]:
df = pd.concat([benign_df, benign_test_df, ddos_df], ignore_index=True)

In [38]:
df = df[[' Fwd Packet Length Max', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', ' Flow Packets/s', 'Bwd IAT Total', 'Fwd PSH Flags', 'Fwd Packets/s', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', ' RST Flag Count', ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count', ' Down/Up Ratio', ' Average Packet Size', ' Avg Fwd Segment Size', ' Avg Bwd Segment Size', ' Init_Win_bytes_backward', ' min_seg_size_forward', 'Idle Mean', ' Idle Min', ' Label']]

In [39]:
X = df.drop([' Label'], axis=1)
y = df[' Label']

In [43]:
for i in range(1, 15):
    clf = OutlierDetection(benign=X, cluster_n=i)
    y_pred_test = clf.predict(X)
    print('Cluster : ' + str(i))
    print('---Classification Report---')
    print(classification_report(y, y_pred_test))

Cluster : 1
---Classification Report---
              precision    recall  f1-score   support

          -1       0.01      0.11      0.01       411
           1       0.97      0.63      0.76     16010

    accuracy                           0.62     16421
   macro avg       0.49      0.37      0.39     16421
weighted avg       0.94      0.62      0.74     16421

Cluster : 2
---Classification Report---
              precision    recall  f1-score   support

          -1       0.01      0.11      0.02       411
           1       0.97      0.64      0.77     16010

    accuracy                           0.63     16421
   macro avg       0.49      0.38      0.39     16421
weighted avg       0.94      0.63      0.75     16421

Cluster : 3
---Classification Report---
              precision    recall  f1-score   support

          -1       0.01      0.10      0.02       411
           1       0.97      0.70      0.81     16010

    accuracy                           0.68     16421
   macro